In [1]:
from datasets import load_from_disk
import evaluate
import math
import multiprocessing as mp
import numpy as np
from ray import tune
import torch
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, default_data_collator

torch.backends.cuda.matmul.allow_tf32 = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: %s" % device)

FRAMES_PER_VIDEO = 8

Device: cuda


In [2]:
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
tokenizer.pad_token = tokenizer.eos_token

def model_init(trial):
    model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
    model.config.decoder_start_token_id = tokenizer.bos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    return model

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [3]:
# TimeSformer
# encoder = "facebook/timesformer-base-finetuned-k600"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

# tokenizer.pad_token = tokenizer.eos_token
# model.config.decoder_start_token_id = tokenizer.bos_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# VideoMAE
# encoder = "MCG-NJU/videomae-base"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained(encoder)
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)

In [4]:
dataset = load_from_disk("/home/922201615/video-caption/dataset/processed/8frames_pt1")
dataset.set_format(type="torch")

train_idxs = np.linspace(0, len(dataset["train"]) - 1, num=len(dataset["train"]) // 20).astype(np.int64)
val_idxs = np.linspace(0, len(dataset["validation"]) - 1, num=len(dataset["validation"]) // 20).astype(np.int64)

# train_idxs = [x for x in range(40)]
# val_idxs = [x for x in range(40, 44)]

dataset["train"] = dataset["train"].select(train_idxs)
dataset["validation"] = dataset["validation"].select(val_idxs)
dataset

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 1144
    })
    validation: Dataset({
        features: ['pixel_values', 'labels'],
        num_rows: 132
    })
})

In [5]:
# metric = evaluate.load("rouge")
# def metrics(eval_preds):
#     preds, labels = eval_preds
#     preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     return metric.compute(predictions=preds, references=labels)

training_args = Seq2SeqTrainingArguments(
    output_dir="training/hp_tuning",
    predict_with_generate=True,
    tf32=True,
    dataloader_num_workers=4,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="steps",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

# trainer = Seq2SeqTrainer(
#     model=model,
#     tokenizer=image_processor,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["validation"],
#     data_collator=default_data_collator,
#     compute_metrics=metrics,
# )

# trainer.train()

trainer = Seq2SeqTrainer(
    model=None,
    tokenizer=image_processor,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=default_data_collator,
    model_init=model_init,
)

loading configuration file config.json from cache at /home/922201615/.cache/huggingface/hub/models--facebook--timesformer-base-finetuned-k600/snapshots/8b0484b3e11f898a3ef4ddd8ee3db327e288bb27/config.json
Model config TimesformerConfig {
  "_name_or_path": "facebook/timesformer-base-finetuned-k600",
  "architectures": [
    "TimesformerForVideoClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "divided_space_time",
  "drop_path_rate": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "abseiling",
    "1": "acting in play",
    "2": "adjusting glasses",
    "3": "air drumming",
    "4": "alligator wrestling",
    "5": "answering questions",
    "6": "applauding",
    "7": "applying cream",
    "8": "archaeological excavation",
    "9": "archery",
    "10": "arguing",
    "11": "arm wrestling",
    "12": "arranging flowers",
    "13": "assembling bicycle",
    "14": "assembling computer",
    "15": "attend

In [6]:
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "lr_scheduler_type": tune.choice(["linear", "cosine"]),
        "warmup_ratio": tune.uniform(0.0, 0.2),
        "weight_decay": tune.uniform(0.0, 1e-3),
    }

best_trial = trainer.hyperparameter_search(
    resources_per_trial={"cpu": mp.cpu_count(), "gpu": torch.cuda.device_count()},
    direction="minimize",
    backend="ray",
    hp_space=ray_hp_space,
    n_trials=1,
)

best_trial

2023-03-05 20:26:32,412	INFO worker.py:1553 -- Started a local Ray instance.
/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py:612: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(


== Status ==
Current time: 2023-03-05 20:26:34 (running for 00:00:00.08)
Memory usage on this node: 21.2/251.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 64.0/64 CPUs, 1.0/1 GPUs, 0.0/147.5 GiB heap, 0.0/67.2 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/922201615/ray_results/_objective_2023-03-05_20-26-34
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+----------------------+-----------------+---------------------+----------------+
| Trial name             | status   | loc                  |   learning_rate | lr_scheduler_type   |   warmup_ratio |
|------------------------+----------+----------------------+-----------------+---------------------+----------------|
| _objective_13ff3_00000 | RUNNING  | 130.212.82.53:302157 |     5.61152e-06 | linear              |       0.036687 |
+------------------------+----------+----------------------+-----------------+---------------------+----------------+




(_objective pid=302157) Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.bias', 'classifier.weight']
(_objective pid=302157) - This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=302157) - This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=302157) Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.5.crossattention.c_proj.weight', 'h.8.crossattention.bias', 'h.5.crossattention.bias', 'h.6.crossattention.c_proj.bias', 'h.2.crossattention.masked

== Status ==
Current time: 2023-03-05 20:26:41 (running for 00:00:07.09)
Memory usage on this node: 23.8/251.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 64.0/64 CPUs, 1.0/1 GPUs, 0.0/147.5 GiB heap, 0.0/67.2 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/922201615/ray_results/_objective_2023-03-05_20-26-34
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+----------------------+-----------------+---------------------+----------------+
| Trial name             | status   | loc                  |   learning_rate | lr_scheduler_type   |   warmup_ratio |
|------------------------+----------+----------------------+-----------------+---------------------+----------------|
| _objective_13ff3_00000 | RUNNING  | 130.212.82.53:302157 |     5.61152e-06 | linear              |       0.036687 |
+------------------------+----------+----------------------+-----------------+---------------------+----------------+




(_objective pid=302157) /home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(_objective pid=302157)   warnings.warn(
  0%|          | 0/858 [00:00<?, ?it/s]


...

(_objective pid=302157) 
 76%|███████▌  | 25/33 [00:26<00:11,  1.39s/it]
(_objective pid=302157) 
 79%|███████▉  | 26/33 [00:27<00:07,  1.06s/it]
(_objective pid=302157) 
 82%|████████▏ | 27/33 [00:27<00:04,  1.20it/s]
(_objective pid=302157) 
 85%|████████▍ | 28/33 [00:27<00:03,  1.49it/s]
(_objective pid=302157) 
 88%|████████▊ | 29/33 [00:30<00:05,  1.31s/it]
(_objective pid=302157) 
 91%|█████████ | 30/33 [00:30<00:03,  1.01s/it]


== Status ==
Current time: 2023-03-05 20:46:10 (running for 00:19:36.07)
Memory usage on this node: 25.4/251.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 64.0/64 CPUs, 1.0/1 GPUs, 0.0/147.5 GiB heap, 0.0/67.2 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/922201615/ray_results/_objective_2023-03-05_20-26-34
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+----------------------+-----------------+---------------------+----------------+-------------+
| Trial name             | status   | loc                  |   learning_rate | lr_scheduler_type   |   warmup_ratio |   objective |
|------------------------+----------+----------------------+-----------------+---------------------+----------------+-------------|
| _objective_13ff3_00000 | RUNNING  | 130.212.82.53:302157 |     5.61152e-06 | linear              |       0.036687 |   0.0766502 |
+------------------------+----------+----------------------+-----------------+------------------

(_objective pid=302157) 
 94%|█████████▍| 31/33 [00:31<00:01,  1.26it/s]
(_objective pid=302157) 
 97%|█████████▋| 32/33 [00:31<00:00,  1.55it/s]
(_objective pid=302157) 
100%|██████████| 33/33 [00:33<00:00,  1.09s/it]
2023-03-05 20:46:13,299	INFO tune.py:798 -- Total run time: 1178.68 seconds (1178.66 seconds for the tuning loop).


== Status ==
Current time: 2023-03-05 20:46:13 (running for 00:19:38.66)
Memory usage on this node: 23.9/251.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/64 CPUs, 0/1 GPUs, 0.0/147.5 GiB heap, 0.0/67.2 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /home/922201615/ray_results/_objective_2023-03-05_20-26-34
Number of trials: 1/1 (1 TERMINATED)
+------------------------+------------+----------------------+-----------------+---------------------+----------------+-------------+
| Trial name             | status     | loc                  |   learning_rate | lr_scheduler_type   |   warmup_ratio |   objective |
|------------------------+------------+----------------------+-----------------+---------------------+----------------+-------------|
| _objective_13ff3_00000 | TERMINATED | 130.212.82.53:302157 |     5.61152e-06 | linear              |       0.036687 |   0.0762246 |
+------------------------+------------+----------------------+-----------------+----------

BestRun(run_id='13ff3_00000', objective=0.07622462511062622, hyperparameters={'learning_rate': 5.61151641533451e-06, 'lr_scheduler_type': 'linear', 'warmup_ratio': 0.03668695797323276})